In [1]:
# 기본 패키지 임포트
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import trimesh, pickle
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 디바이스 설정 (GPU 사용 가능 시 GPU로 설정)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.enabled = True


C:\ProgramData\anaconda3\envs\project2025\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


In [2]:
import os
from torch.utils.data import Dataset, DataLoader

class LungNoduleDataset(Dataset):
    def __init__(self, data_dir, split='train'):
        self.mesh_files = []
        self.face_labels = []   # 각 메쉬의 face 레이블 경로 또는 데이터
        self.class_labels = []  # 결절 존재 유무 (0 또는 1)
        # 데이터 디렉터리에서 파일 목록 구성
        mesh_dir = os.path.join(data_dir, split, "mesh")
        label_dir = os.path.join(data_dir, split, "label")  # face별 레이블 파일 (예: .npy 또는 .txt)
        for fname in os.listdir(mesh_dir):
            if fname.endswith('.obj') or fname.endswith('.stl') or fname.endswith('.ply'):
                mesh_path = os.path.join(mesh_dir, fname)
                label_path = os.path.join(label_dir, fname.split('.')[0] + "_label.npy")
                self.mesh_files.append(mesh_path)
                if os.path.exists(label_path):
                    face_label = np.load(label_path)  # face 레이블 (예: 0/1 값의 배열)
                else:
                    face_label = None
                self.face_labels.append(face_label)
                # 클래스 레이블: face_label에 1이 하나라도 있으면 1 (결절 있음), 아니면 0
                class_label = 1 if (face_label is not None and face_label.sum() > 0) else 0
                self.class_labels.append(class_label)
    
    def __len__(self):
        return len(self.mesh_files)
    
    def __getitem__(self, idx):
        # 캐시 디렉토리 경로 (문자열로 지정)
        CACHE_DIR = "C:/Users/konyang/Desktop/MeshCNN_TF/data/cache/train"
        os.makedirs(CACHE_DIR, exist_ok=True)  # 디렉토리가 없다면 생성

        # 메쉬 파일 경로와 캐시 파일 경로 생성
        mesh_path = self.mesh_files[idx]
        mesh_stem = os.path.splitext(os.path.basename(mesh_path))[0]  # 파일명에서 확장자 제거
        cache_path = os.path.join(CACHE_DIR, mesh_stem + ".pkl")

        # 캐시가 있다면 로드
        if os.path.exists(cache_path):
            with open(cache_path, 'rb') as f:
                return pickle.load(f)
        
        # 1. 메쉬 파일 로드
        mesh = trimesh.load(self.mesh_files[idx])
        vertices = np.array(mesh.vertices)        # (V, 3) 배열
        faces = np.array(mesh.faces)              # (F, 3) 배열
        face_normals = np.array(mesh.face_normals)  # 각 face의 법선 벡터
        face_labels = self.face_labels[idx]
        
        # 2. 간선 및 간선->면 매핑 생성
        edge_to_faces = {}
        for f_idx, face in enumerate(faces):
            # 삼각형 face의 세 변 (정렬하여 tuple로 사용)
            for e in [(face[0], face[1]), (face[1], face[2]), (face[2], face[0])]:
                e_sorted = tuple(sorted(e))
                if e_sorted not in edge_to_faces:
                    edge_to_faces[e_sorted] = []
                edge_to_faces[e_sorted].append(f_idx)
        edges = list(edge_to_faces.keys())               # 간선 리스트 (고유 간선)
        
        # 3. 간선 이웃 정보 계산 (각 간선당 최대 4개 이웃 간선)
        # 이웃 간선 정의: 하나의 꼭짓점을 공유하는 간선 (1-링 이웃)
        vert_to_edges = {v: [] for v in range(len(vertices))}
        for e_idx, (v1, v2) in enumerate(edges):
            vert_to_edges[v1].append(e_idx)
            vert_to_edges[v2].append(e_idx)
        neighbors_list = []
        for e_idx, (v1, v2) in enumerate(edges):
            neighbor_set = set(vert_to_edges[v1] + vert_to_edges[v2])
            neighbor_set.discard(e_idx)  # 자기 자신은 제외
            neighbors = list(neighbor_set)
            # 최대 4개까지 이웃 간선을 선택 (많으면 자르기)
            neighbors = neighbors[:4]
            # 4개 미만이면 자기 자신으로 패딩하여 크기 고정
            while len(neighbors) < 4:
                neighbors.append(e_idx)
            neighbors_list.append(neighbors)
        neighbor_index = torch.tensor(neighbors_list, dtype=torch.long)
        
        # 4. 간선 특징 계산 (5차원: dihedral, inner1, inner2, ratio1, ratio2)
        edge_features = []
        for e_idx, (v1, v2) in enumerate(edges):
            p1, p2 = vertices[v1], vertices[v2]
            # 간선 길이
            edge_length = np.linalg.norm(p1 - p2)
            # 간선을 공유하는 면 목록
            faces_indices = edge_to_faces[(v1, v2)]
            # Dihedral angle 계산
            if len(faces_indices) == 2:
                f1, f2 = faces_indices[0], faces_indices[1]
                n1, n2 = face_normals[f1], face_normals[f2]
                cos_theta = np.dot(n1, n2) / (np.linalg.norm(n1)*np.linalg.norm(n2) + 1e-8)
                cos_theta = np.clip(cos_theta, -1.0, 1.0)
                dihedral = np.arccos(cos_theta)
            else:
                # 인접 면이 하나뿐인 경우 (경계 간선) - dihedral을 0으로 처리
                dihedral = 0.0
            
            # 첫 번째 면의 대향각 및 비율
            inner1 = 0.0; ratio1 = 0.0
            if len(faces_indices) > 0:
                f1 = faces_indices[0]
                # 간선의 반대쪽 꼭짓점
                face_v = faces[f1]
                # 간선을 이루는 두 꼭짓점을 제외한 나머지 한 꼭짓점
                opp_v = [v for v in face_v if v not in (v1, v2)][0]
                vec1 = vertices[v1] - vertices[opp_v]
                vec2 = vertices[v2] - vertices[opp_v]
                cos_inner = np.dot(vec1, vec2) / ((np.linalg.norm(vec1)*np.linalg.norm(vec2)) + 1e-8)
                cos_inner = np.clip(cos_inner, -1.0, 1.0)
                inner1 = np.arccos(cos_inner)
                # 높이 계산 (삼각형 면적 이용)
                area = np.linalg.norm(np.cross(vec1, vec2)) / 2.0
                height = (2.0 * area) / (edge_length + 1e-8)
                ratio1 = edge_length / (height + 1e-8)
            # 두 번째 면의 대향각 및 비율 (없으면 첫 번째 면 값으로 대체)
            inner2 = inner1; ratio2 = ratio1
            if len(faces_indices) == 2:
                f2 = faces_indices[1]
                face_v2 = faces[f2]
                opp_v2 = [v for v in face_v2 if v not in (v1, v2)][0]
                vec3 = vertices[v1] - vertices[opp_v2]
                vec4 = vertices[v2] - vertices[opp_v2]
                cos_inner2 = np.dot(vec3, vec4) / ((np.linalg.norm(vec3)*np.linalg.norm(vec4)) + 1e-8)
                cos_inner2 = np.clip(cos_inner2, -1.0, 1.0)
                inner2 = np.arccos(cos_inner2)
                area2 = np.linalg.norm(np.cross(vec3, vec4)) / 2.0
                height2 = (2.0 * area2) / (edge_length + 1e-8)
                ratio2 = edge_length / (height2 + 1e-8)
            # 특징 벡터 구성
            edge_features.append([dihedral, inner1, inner2, ratio1, ratio2])
        
        edge_features = torch.tensor(edge_features, dtype=torch.float32)
        
        # 5. 분할 레이블 (간선 단위) 생성
        if face_labels is not None:
            face_labels_arr = face_labels  # numpy array of shape (F,)
            edge_labels = []
            for e_idx, (v1, v2) in enumerate(edges):
                faces_indices = edge_to_faces[(v1, v2)]
                # 해당 간선에 인접한 face 중 하나라도 결절(1)인 경우 간선 레이블 1
                label = 0
                for f_idx in faces_indices:
                    if face_labels_arr[f_idx] == 1:
                        label = 1
                        break
                edge_labels.append(label)
            edge_labels = torch.tensor(edge_labels, dtype=torch.long)
        else:
            # face 레이블이 없는 경우 (분할 라벨이 없으면 모두 0으로 처리)
            edge_labels = torch.zeros(len(edges), dtype=torch.long)
        
        # 6. 클래스 레이블 (결절 유무)
        class_label = torch.tensor(self.class_labels[idx], dtype=torch.long)
        
        # 결과 반환: 특징, 이웃정보, 클래스 레이블, 간선 레이블, (필요하면 원본 메쉬 정보도 반환 가능)
        sample = {
            'edge_features': edge_features,
            'neighbor_index': neighbor_index,
            'class_label': class_label,
            'edge_labels': edge_labels,
            'vertices': vertices,   # 시각화를 위해 전달
            'faces': faces          # 시각화를 위해 전달
        }
        
        with open(cache_path, 'wb') as f:
            pickle.dump(sample, f)

        return sample

# 데이터셋 및 데이터로더 초기화 예시 (파일 경로는 실제 데이터셋에 맞게 수정해야 함)
data_dir = "C:/Users/konyang/Desktop/MeshCNN_TF/data"
train_dataset = LungNoduleDataset(data_dir, split='train')
train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)


In [3]:
class MeshConv(nn.Module):
    """메쉬 간선 합성곱 레이어: 각 간선과 이웃한 4개 간선 (총5개)의 특징을 통합해 출력."""
    def __init__(self, in_channels, out_channels):
        super(MeshConv, self).__init__()
        # 5 * in_channels 크기의 입력을 out_channels로 변환
        self.linear = nn.Linear(in_channels * 5, out_channels)
    
    def forward(self, x, neighbor_index):
        # x: (E, in_channels) 간선 특징
        # neighbor_index: (E, 4) 각 간선의 이웃 간선 인덱스
        # 이웃 간선의 특징 수집
        # x_neighbors: (E, 4, in_channels)
        x_neighbors = x[neighbor_index]  
        # x_self: (E, 1, in_channels)
        x_self = x.unsqueeze(1)  
        # 자신 + 이웃을 concatenation: (E, 5, in_channels)
        x_combined = torch.cat([x_self, x_neighbors], dim=1)
        # 펼쳐서 Linear 입력으로: (E, 5*in_channels)
        x_flat = x_combined.view(x_combined.size(0), -1)
        # Linear 변환
        out = self.linear(x_flat)
        return out  # 출력 크기: (E, out_channels)

class MeshPool(nn.Module):
    """메쉬 풀링 레이어: 간선 수를 감소 (2:1 비율로 클러스터)."""
    def __init__(self):
        super(MeshPool, self).__init__()
        self.cluster_map = None  # 나중에 unpool에 사용하기 위한 매핑
    
    def forward(self, x, neighbor_index):
        # x: (N, C) 입력 간선 특징, N = 현재 간선 수
        N = x.size(0)
        # 클러스터 구성: 2개 간선씩 묶음 (홀수개일 경우 마지막은 단독 클러스터)
        if N % 2 == 0:
            new_N = N // 2
        else:
            new_N = N // 2 + 1
        # 출력 텐서 초기화
        device = x.device
        C = x.size(1)
        x_pooled = torch.zeros((new_N, C), dtype=x.dtype, device=device)
        # cluster_map: 길이 N 리스트로, 각 간선이 속한 클러스터 인덱스
        cluster_map = [-1] * N
        # 2개씩 평균
        for i in range(N // 2):
            idx1 = 2 * i
            idx2 = 2 * i + 1
            x_pooled[i] = 0.5 * (x[idx1] + x[idx2])
            cluster_map[idx1] = i
            cluster_map[idx2] = i
        if N % 2 == 1:
            # 마지막 간선은 단독 클러스터
            x_pooled[new_N - 1] = x[N - 1]
            cluster_map[N - 1] = new_N - 1
        # cluster_map을 Tensor로 저장
        cluster_map = torch.tensor(cluster_map, dtype=torch.long, device=device)
        self.cluster_map = cluster_map
        
        # 새로운 neighbor_index 계산 (클러스터 그래프의 이웃)
        neighbors_coarse = []
        # 원래 간선의 neighbor_index로부터 클러스터 간 이웃을 유추
        for edge_idx in range(N):
            cluster_idx = cluster_map[edge_idx].item()
            # 원래 간선의 이웃들의 클러스터 인덱스
            for nbr_edge in neighbor_index[edge_idx]:
                nbr_cluster = cluster_map[nbr_edge].item()
                if nbr_cluster != cluster_idx:
                    # 리스트 길이를 cluster_idx에 맞춰 확장
                    if cluster_idx >= len(neighbors_coarse):
                        neighbors_coarse.extend([set() for _ in range(cluster_idx - len(neighbors_coarse) + 1)])
                    neighbors_coarse[cluster_idx].add(nbr_cluster)
        # 집합을 리스트로 변환하고 4개로 패딩
        for i in range(len(neighbors_coarse)):
            nbrs = list(neighbors_coarse[i])
            nbrs = nbrs[:4]  # 최대 4개까지 사용
            while len(nbrs) < 4:
                nbrs.append(i)  # 자기 자신으로 패딩
            neighbors_coarse[i] = nbrs
        # 만약 어떤 클러스터에 대해 neighbor_set이 비어있으면 자기 자신 4개로
        if len(neighbors_coarse) < new_N:
            # 빈 클러스터 neighbor 세트 초기화
            for i in range(len(neighbors_coarse), new_N):
                neighbors_coarse.append([i, i, i, i])
        neighbor_index_coarse = torch.tensor(neighbors_coarse, dtype=torch.long, device=device)
        
        return x_pooled, neighbor_index_coarse

class MeshUnpool(nn.Module):
    """메쉬 업풀링 레이어: 풀링된 간선 특징을 원래 개수로 복원."""
    def forward(self, x_coarse, cluster_map):
        # x_coarse: (M, C) 풀링된 간선 특징, cluster_map: (N,) 각 원래 간선 -> 클러스터 인덱스 매핑
        device = x_coarse.device
        N = cluster_map.shape[0]  # 원래 간선 개수
        C = x_coarse.shape[1]
        # cluster_map을 이용해 각 원래 간선의 특징 할당
        x_reconstructed = torch.zeros((N, C), dtype=x_coarse.dtype, device=device)
        # cluster_map의 각 인덱스를 순회하며 할당
        for orig_edge_idx, cluster_idx in enumerate(cluster_map):
            x_reconstructed[orig_edge_idx] = x_coarse[cluster_idx]
        return x_reconstructed

# 이제 Encoder, Decoder, Classifier, Segmenter를 정의
class MeshEncoder(nn.Module):
    def __init__(self, in_channels=5):
        super(MeshEncoder, self).__init__()
        # 채널 설정: conv1_out=16, conv2_out=32, conv3_out=64 (예시)
        self.conv1 = MeshConv(in_channels, 16)
        self.pool1 = MeshPool()
        self.conv2 = MeshConv(16, 32)
        self.pool2 = MeshPool()
        self.conv3 = MeshConv(32, 64)
    
    def forward(self, x, neighbor_index):
        # conv1 + ReLU
        x1 = F.relu(self.conv1(x, neighbor_index))
        # pool1
        x_pooled1, neighbor_coarse1 = self.pool1(x1, neighbor_index)
        # conv2 + ReLU on pooled1
        x2 = F.relu(self.conv2(x_pooled1, neighbor_coarse1))
        # pool2
        x_pooled2, neighbor_coarse2 = self.pool2(x2, neighbor_coarse1)
        # conv3 + ReLU on pooled2 (encoder 최종)
        x3 = F.relu(self.conv3(x_pooled2, neighbor_coarse2))
        # Encoder 결과와 중간 결과 반환 (스킵 연결 및 Decoder에 필요)
        return {
            'x1': x1,                       # original level features (E edges, 16ch)
            'neighbor0': neighbor_index, 
            'x2': x2,                       # half level features (E/2 edges, 32ch)
            'neighbor1': neighbor_coarse1,
            'x3': x3,                       # quarter level features (E/4 edges, 64ch)
            'neighbor2': neighbor_coarse2,
            'pool1': self.pool1,
            'pool2': self.pool2
        }

class MeshDecoder(nn.Module):
    def __init__(self):
        super(MeshDecoder, self).__init__()
        # Decoder conv layers (업샘플 후 특징 합성)
        # conv2_up: 입력 채널 = 32(skip) + 64(decoder up) = 96, 출력 32
        self.conv2_up = MeshConv(32 + 64, 32)
        # conv1_up: 입력 채널 = 16(skip) + 32(decoder up) = 48, 출력 16
        self.conv1_up = MeshConv(16 + 32, 16)
    
    def forward(self, enc_out):
        # enc_out: Encoder의 출력 딕셔너리
        x2 = enc_out['x2']
        x3 = enc_out['x3']
        neighbor1 = enc_out['neighbor1']
        neighbor0 = enc_out['neighbor0']
        pool1_layer = enc_out['pool1']
        pool2_layer = enc_out['pool2']
        
        # 1. 업풀링 두 번째 풀 (quarter -> half)
        x2_up = MeshUnpool().forward(x3, pool2_layer.cluster_map)  # quarter->half 복원 (32ch)
        # 2. 스킵 연결 결합 (half 해상도): Encoder의 x2와 concat
        x2_cat = torch.cat([x2, x2_up], dim=1)  # shape: (E/2, 96)
        # 3. conv2_up + ReLU (half 해상도 특징 복원)
        x2_up_refined = F.relu(self.conv2_up(x2_cat, neighbor1))
        # 4. 업풀링 첫 번째 풀 (half -> original)
        x1_up = MeshUnpool().forward(x2_up_refined, pool1_layer.cluster_map)  # half->original 복원 (16ch+)
        # 5. 스킵 연결 결합 (original 해상도): Encoder의 x1과 concat
        x1 = enc_out['x1']
        x1_cat = torch.cat([x1, x1_up], dim=1)  # shape: (E, 48)
        # 6. conv1_up + ReLU (original 해상도 특징 복원)
        x1_up_refined = F.relu(self.conv1_up(x1_cat, neighbor0))
        return x1_up_refined  # (E, 16) 원래 간선 수의 복원된 특징

class ClassificationHead(nn.Module):
    def __init__(self, in_channels, hidden_dim=100, num_classes=2):
        super(ClassificationHead, self).__init__()
        # 단순한 MLP 분류기: [in_channels] -> [hidden_dim] -> [num_classes]
        self.fc1 = nn.Linear(in_channels, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, num_classes)
    
    def forward(self, x):
        # x: (in_channels) or (1, in_channels) 전역 특징 벡터
        x = F.relu(self.fc1(x))
        out = self.fc2(x)
        return out  # raw logits (num_classes)
    
class SegmentationHead(nn.Module):
    def __init__(self, in_channels, num_classes=2):
        super(SegmentationHead, self).__init__()
        self.linear = nn.Linear(in_channels, num_classes)
    def forward(self, x):
        # x: (E, in_channels) 모든 원래 간선에 대한 복원된 특징
        out = self.linear(x)  # (E, num_classes)
        return out

# 전체 모델 통합
class MedMeshNet(nn.Module):
    def __init__(self):
        super(MedMeshNet, self).__init__()
        self.encoder = MeshEncoder(in_channels=5)
        self.decoder = MeshDecoder()
        # classification은 encoder 최종 출력 채널 (예: 64)을 받아 이진 분류
        self.classifier = ClassificationHead(in_channels=64, hidden_dim=100, num_classes=2)
        # segmentation은 decoder 최종 출력 채널 (예: 16)을 받아 2-class 출력
        self.segmenter = SegmentationHead(in_channels=16, num_classes=2)
    
    def forward(self, edge_features, neighbor_index):
        # 1. Encoder: 특징 추출 및 다운샘플
        enc_out = self.encoder(edge_features, neighbor_index)
        # 2. Classification: Encoder 최종 특징들을 전역 요약하여 클래스 예측
        x_global = torch.max(enc_out['x3'], dim=0)[0]  # 전역 max 풀 (64차원 벡터)
        class_logits = self.classifier(x_global.unsqueeze(0))  # (1,2) 출력
        # 3. Decoder: 업샘플로 원래 해상도 특징 복원
        dec_out = self.decoder(enc_out)  # (E, 16)
        seg_logits = self.segmenter(dec_out)  # (E, 2)
        return class_logits, seg_logits


In [4]:
# 모델 초기화
model = MedMeshNet().to(device)

# 손실 함수 정의
criterion_class = nn.CrossEntropyLoss()  # 분류 손실
criterion_seg = nn.CrossEntropyLoss()    # 분할 손실 (다중 클래스용)

# 옵티마이저 & 스케줄러 설정
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# 검증용 val_loader 존재 확인
try:
    val_loader
except NameError:
    val_loader = None

# 학습 루프
num_epochs = 10
for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0.0

    for batch in train_loader:
        features = batch['edge_features'].to(device)       # (E, 5)
        neighbor_index = batch['neighbor_index'].to(device)  # (E, 4)
        class_label = batch['class_label'].to(device)      # (B,)
        seg_label = batch['edge_labels'].to(device)        # (E,)

        # 디버깅 로그
        if epoch == 1:
            print("features:", features.shape)
            print("neighbor_index:", neighbor_index.shape)
            print("neighbor_index min:", neighbor_index.min().item())
            print("neighbor_index max:", neighbor_index.max().item())

        # 순전파
        class_logits, seg_logits = model(features, neighbor_index)  # class_logits: (B, C), seg_logits: (E, C)

        # 손실 계산
        loss_class = criterion_class(class_logits, class_label)
        loss_seg = criterion_seg(seg_logits, seg_label)
        loss = loss_class + loss_seg

        # 역전파 및 최적화
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    print(f"[Epoch {epoch}] 평균 훈련 손실: {avg_loss:.4f}")

    # 스케줄러 업데이트
    scheduler.step()

    # ---------------------------
    # 검증 단계
    # ---------------------------
    if val_loader is not None:
        model.eval()
        correct = 0
        total_samples = 0
        inter = 0
        union = 0

        with torch.no_grad():
            for batch in val_loader:
                features = batch['edge_features'].to(device)
                neighbor_index = batch['neighbor_index'].to(device)
                class_label = batch['class_label'].to(device)
                seg_label = batch['edge_labels'].to(device)

                # 순전파
                class_logits, seg_logits = model(features, neighbor_index)

                # 분류 정확도
                pred_class = torch.argmax(class_logits, dim=1)
                correct += (pred_class == class_label).sum().item()
                total_samples += class_label.size(0)

                # 분할 IoU (다중 클래스 대응)
                pred_seg = torch.argmax(seg_logits, dim=1)
                num_classes = seg_logits.shape[1]

                for cls in range(num_classes):
                    inter += ((pred_seg == cls) & (seg_label == cls)).sum().item()
                    union += ((pred_seg == cls) | (seg_label == cls)).sum().item()

        val_acc = correct / total_samples if total_samples > 0 else 0
        val_iou = inter / (union + 1e-8) if union > 0 else 0
        print(f"  - 검증 정확도: {val_acc * 100:.2f}%, 검증 IoU: {val_iou:.4f}")

features: torch.Size([1, 744127, 5])
neighbor_index: torch.Size([1, 744127, 4])
neighbor_index min: 0
neighbor_index max: 744126


RuntimeError: CUDA out of memory. Tried to allocate 41255.73 GiB (GPU 0; 8.00 GiB total capacity; 42.73 MiB already allocated; 5.93 GiB free; 62.00 MiB reserved in total by PyTorch)

In [ ]:
def compute_classification_accuracy(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in data_loader:
            features = batch['edge_features'].to(device)
            neighbor_index = batch['neighbor_index'].to(device)
            labels = batch['class_label'].to(device)
            class_logits, _ = model(features, neighbor_index)
            pred = torch.argmax(class_logits, dim=1)  # 예측 클래스
            if pred.item() == labels.item():
                correct += 1
            total += 1
    acc = correct / total if total > 0 else 0
    return acc

def compute_segmentation_iou(model, data_loader):
    model.eval()
    inter = 0
    union = 0
    with torch.no_grad():
        for batch in data_loader:
            features = batch['edge_features'].to(device)
            neighbor_index = batch['neighbor_index'].to(device)
            true_seg = batch['edge_labels'].to(device)
            _, seg_logits = model(features, neighbor_index)
            pred_seg = torch.argmax(seg_logits, dim=1)
            # 결절 클래스(1)에 대한 IoU 계산
            inter += torch.logical_and(pred_seg == 1, true_seg == 1).sum().item()
            union += torch.logical_or(pred_seg == 1, true_seg == 1).sum().item()
    iou = inter / (union + 1e-8) if union > 0 else 0
    return iou

# 예시: 학습 완료 후 테스트 세트에 대한 성능 측정

test_acc = compute_classification_accuracy(model, test_loader)
test_iou = compute_segmentation_iou(model, test_loader)
print(f"테스트 정확도: {test_acc:.4f}, 테스트 IoU: {test_iou:.4f}")


In [ ]:
# 예측 및 시각화 예시 (단일 샘플)
model.eval()
sample = val_dataset[0]  # 검증 세트 첫 번째 샘플 (예시)
features = sample['edge_features'].to(device)
neighbor_index = sample['neighbor_index'].to(device)
vertices = sample['vertices']  # numpy array
faces = sample['faces']        # numpy array

# 모델 예측
class_logits, seg_logits = model(features, neighbor_index)
pred_class = torch.argmax(class_logits, dim=1).item()  # 0 또는 1
pred_seg = torch.argmax(seg_logits, dim=1).cpu().numpy()  # 각 간선의 예측 클래스 (numpy로 변환)

# 결과 출력 - 분류
if pred_class == 1:
    print("모델 예측: 이 메쉬에서 폐 결절이 검출되었습니다.")
else:
    print("모델 예측: 이 메쉬에는 결절이 없습니다.")

# 시각화를 위해 간선->면 매핑 다시 계산 (sample 생성 시 활용 가능)
edge_to_faces = {}
for f_idx, face in enumerate(faces):
    for e in [(face[0], face[1]), (face[1], face[2]), (face[2], face[0])]:
        e_sorted = tuple(sorted(e))
        if e_sorted not in edge_to_faces:
            edge_to_faces[e_sorted] = []
        edge_to_faces[e_sorted].append(f_idx)
edges = list(edge_to_faces.keys())

# 예측된 결절 간선들에 인접한 face 수집
pred_nodule_faces = set()
for e_idx, edge in enumerate(edges):
    if pred_seg[e_idx] == 1:  # 간선이 결절로 예측된 경우
        for f_idx in edge_to_faces[edge]:
            pred_nodule_faces.add(f_idx)

pred_nodule_faces = list(pred_nodule_faces)
print(f"예측된 결절 영역에 포함된 면 개수: {len(pred_nodule_faces)}개")

# 메쉬 객체 생성 (trimesh) 및 face 색상 지정
mesh = trimesh.Trimesh(vertices=vertices, faces=faces)
# 기본 face 색을 회색으로
face_colors = np.tile([200, 200, 200, 255], (len(faces), 1))  # RGBA
# 결절 예측 faces를 빨간색으로 칠함
for f_idx in pred_nodule_faces:
    face_colors[f_idx] = [255, 0, 0, 255]  # 빨간색 RGBA
mesh.visual.face_colors = face_colors

# matplotlib 3D 출력
font_location = 'C:/Windows/Fonts/HANDotum.ttf'
font_name = fm.FontProperties(fname=font_location).get_name()
plt.rc('font', family=font_name)
plt.rcParams['axes.unicode_minus'] = False

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(111, projection='3d')
# 각 face를 폴리곤으로 plot
for f_idx, face in enumerate(mesh.faces):
    tri_coords = mesh.vertices[face]  # (3,3) 좌표
    # 삼각형을 폴리곤으로 추가
    tri = plt.Polygon(tri_coords[:, :2], closed=True, facecolor=face_colors[f_idx][:3]/255, edgecolor=None)
    # Note: 위에서는 2D 평면에 투영해서 그리므로 z좌표 무시 (간단화를 위해)
ax.add_collection3d(plt.PolyCollection([mesh.vertices[face]], facecolors=face_colors[f_idx][:3]/255, linewidths=0.1, edgecolors='k', alpha=0.9))
# 위의 방식은 단순화된 예이며, 보다 나은 3D 시각화를 위해서는 pyvista 등 사용 가능
plt.title("예측된 결절 부위 시각화")
plt.show()


하이퍼파라미터 조정, 데이터 증강, 평가 지표 추가 및 최적화 등 추가 작업 필요